In [1]:
%pip install google-generativeai
%pip install pillow

import google.generativeai as genai

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 15.3 MB/s eta 0:00:000:00:0136m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.3 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl (9.3 kB)
Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached rsa-4.9-py3-none-any.whl (34 kB)
Using cached uritemplate-4.1.1-py2.py3-none-any.

/Users/vitou/workspace/inception-studio/screenshot-thing/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
programs = genai.protos.Schema(
    type=genai.protos.Type.OBJECT,
    properties={
        'program_name': genai.protos.Schema(type=genai.protos.Type.STRING),
        'program_description': genai.protos.Schema(
            type=genai.protos.Type.STRING,
            description="Description of the program"
        ),
        'is_active': genai.protos.Schema(
            type=genai.protos.Type.BOOLEAN,
            description="Whether the program is currently active"
        ),
        'program_type': genai.protos.Schema(
            type=genai.protos.Type.STRING,
            enum=[
                "browser",
                "email",
                "chat",
                "calendar",
                "other"
            ],
            description="Type of the program"
        ),
        'program_info': genai.protos.Schema(
            type=genai.protos.Type.STRING,
            description="valid json information about the program based on the program type, e.g email client with email showing should have subject, content, sender, etc. if it is a browser it will be the url, page title, etc. if it is a calendar it will be the date, time, highlighted, conflict"
        ),

    },
    required=['program_name', 'program_description', 'program_type']
)

In [4]:
screenshot_data_schema = genai.protos.Schema(
    type=genai.protos.Type.OBJECT,
    properties={
        'general_scene_description': genai.protos.Schema(
            type=genai.protos.Type.STRING,
            description="Description of screenshot and user activity in the scene"
        ),
        'programs_in_scene': genai.protos.Schema(
            type=genai.protos.Type.ARRAY,
            items=programs,
            description="An array of all programs running in the screenshot it may only be one"
        ),
        'mouse_action': genai.protos.Schema(
            type=genai.protos.Type.STRING,
            description="what the user is doing with the mouse, hovering a button, clicking a button, etc."
        )
    },
    required=['general_scene_description', 'programs_in_scene', 'mouse_action']
)

In [5]:
screenshot_tools = genai.protos.FunctionDeclaration(
    name="screenshot_data",
    description="Get data about the screenshot and what the user is doing with the mouse",
    parameters=screenshot_data_schema
)


In [6]:
import os


api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)


In [7]:
model = genai.GenerativeModel(
    "gemini-1.5-pro-latest",
    tools=[screenshot_tools]
)


In [67]:
import os
from PIL import Image

current_dir = os.getcwd()
image_dir = os.path.join(current_dir, "agent", "final_images")
print(image_dir)

/Users/vitou/workspace/inception-studio/screenshot-thing/agent/final_images


In [72]:
files = sorted(os.listdir(image_dir))

In [73]:
results = []
for file in files:
    image = Image.open(os.path.join(image_dir, file))
    result = model.generate_content(
        ["describe the screenshot and what the user is doing with the mouse", image]
    )
    results.append(result)

In [74]:
# Extract the actions
actions = "# Action History\n\n"
for i, result in enumerate(results):
    for candidate in result.candidates:
        part = candidate.content.parts[0]
        actions += f"## Action {i+1}\n{str(part.function_call)}\n"


In [75]:
actions

'# Action History\n\n## Action 1\nname: "screenshot_data"\nargs {\n  fields {\n    key: "programs_in_scene"\n    value {\n      list_value {\n        values {\n          struct_value {\n            fields {\n              key: "program_type"\n              value {\n                string_value: "email"\n              }\n            }\n            fields {\n              key: "program_name"\n              value {\n                string_value: "Email Client"\n              }\n            }\n            fields {\n              key: "program_description"\n              value {\n                string_value: "Email client"\n              }\n            }\n            fields {\n              key: "is_active"\n              value {\n                bool_value: true\n              }\n            }\n          }\n        }\n      }\n    }\n  }\n  fields {\n    key: "mouse_action"\n    value {\n      string_value: "The mouse cursor is hovering over the email list."\n    }\n  }\n  fields {\n    k

In [78]:
! pip install --upgrade google-cloud-aiplatform

  Using cached google_cloud_storage-2.18.2-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_cloud_core-2.4.1-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached grpc_google_iam_v1-0.13.1-py2.py3-none-any.whl.metadata (3.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 12.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01
Using cached docstring_parser-0.16-py3-none-any.whl (36 kB)
Using cached google_cloud_storage-2.18.2-py2.py3-none-any.whl (130 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.2 MB/s eta 0:00:00
Using cached google_cloud_core-2.4.1-py2.py3-none-any.whl (29 kB)
Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl (81 kB)
Using cached grpc_google_iam_v1-0.13.1-py2.py3-none-any.whl (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 12.6 MB/s eta 0:00:000:00:01m eta 0:00:0

In [85]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting

def generate():
    model = genai.GenerativeModel(
        "gemini-1.5-pro-002",
        system_instruction=["""Your task is to understand how an employee executes a task."""]
    )
    responses = model.generate_content(
        [text1],
        generation_config=generation_config,
        # safety_settings=safety_settings,
        stream=True,
    )

    for response in responses:
        print(response.text, end="")

text1 = """An employee receives an email from a client requesting to change the schedule. The employee goes through a series of actions to complete that task. `# Action History` holds the list of activities for executing this task. Some information in the activity list is not obvious to the newcomers. 

Your task is to comprehend each activity and infer the abstract overview concept, capturing what each task is about, and why each task is performed. Find a series of actions of what has been done. Then, infer the non-obvious in-depth as to WHY each action is taken.  

Extract the reasoning behind each action and decision in bullet points. Think step by step and write it in the #Thinking section. Finally, comprehend key important lessons from this interaction in the #Lesson section.

input: # Action History\\n\\n## Action 1\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"status\\\\\": \\\\\"Syncing\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email client application.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the right panel.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client application. The inbox is empty, and a message displays \\\\\"What a productive day! You\\\\\\\\\\\\\\\'ve accomplished a lot\\\\\". The client is currently syncing.\"\\n    }\\n  }\\n}\\n\\n## Action 2\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Mail\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"Email client application\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse is idle.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client application. The inbox is empty and displays a congratulatory message: \\\\\"What a productive day! You\\\\\\\\\\\\\\\'ve accomplished a lot\\\\\". The application is currently syncing.\"\\n    }\\n  }\\n}\\n\\n## Action 3\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Mail\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"Email client open with no emails present, showing a congratulatory message about having a productive day.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the \\\\\"Sync\\\\\" button.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client with no emails in the inbox. The email client displays a hot air balloon image and the text \\\\\"What a productive day! You\\\\\\\\\\\\\\\'ve accomplished a lot\\\\\".\"\\n    }\\n  }\\n}\\n\\n## Action 4\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"sender\\\\\": \\\\\"Fujitsu\\\\\", \\\\\"subject\\\\\": \\\\\"Rescheduling Meeting\\\\\", \\\\\"content\\\\\": \\\\\"Hi, Can we reschedule our meeting one hour later, from 2024-11-17 1...\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email from Fujitsu regarding rescheduling a meeting.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"Hovering over the Sync button\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client with a request to reschedule a meeting. The user is about to click the Sync button.\"\\n    }\\n  }\\n}\\n\\n## Action 5\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"sender\\\\\": \\\\\"Fujitsu\\\\\", \\\\\"subject\\\\\": \\\\\"Rescheduling Meeting\\\\\", \\\\\"content\\\\\": \\\\\"Hi, Can we reschedule our meeting one hour later, from 2024-11-17 1...\\\\\", \\\\\"time\\\\\": \\\\\"12:59\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email client displaying a message about rescheduling a meeting.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"Hovering over the email content.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client. The user is reading an email about rescheduling a meeting.\"\\n    }\\n  }\\n}\\n\\n## Action 6\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"subject\\\\\": \\\\\"Rescheduling Meeting\\\\\", \\\\\"sender\\\\\": \\\\\"Fujitsu <fujitsu.fujitsuxxx@gmail.com>\\\\\", \\\\\"recipient\\\\\": \\\\\"kta.yasuda@gmail.com\\\\\", \\\\\"body\\\\\": \\\\\"Hi,\\\\\\\\\\\\\\\\n\\\\\\\\\\\\\\\\nCan we reschedule our meeting one hour later,\\\\\\\\\\\\\\\\nfrom 2024-11-17 16:00~17:00 to\\\\\\\\\\\\\\\\n17:00~18:00?\\\\\\\\\\\\\\\\n\\\\\\\\\\\\\\\\nThank you,\\\\\\\\\\\\\\\\nFujitsu\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email client open to a message thread.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the email body text.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client with an email thread open. The user is reading an email about rescheduling a meeting.\"\\n    }\\n  }\\n}\\n\\n## Action 7\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"date\\\\\": \\\\\"November 16, 2024\\\\\", \\\\\"events\\\\\": [{\\\\\"time\\\\\": \\\\\"13:01\\\\\", \\\\\"title\\\\\": \\\\\"Client Meeting: Fujitsu\\\\\"}, {\\\\\"time\\\\\": \\\\\"13:01\\\\\", \\\\\"title\\\\\": \\\\\"Important Meeting with Boss\\\\\"}]}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"A calendar application.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the \\\\\"Client Meeting: Fujitsu\\\\\" entry.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows a calendar application open on a computer. The calendar is displaying the week of November 11th to 17th, 2024.  The current date is Saturday, November 16th. Two meetings are scheduled for Saturday: one with Fujitsu and another, marked as important, with the user\\\\\\\\\\\\\\\'s boss. The current time is 1:01 PM. The location is set to New York, NY, and the weather is 10°C (feels like 16°C) with a low of 7°C.\"\\n    }\\n  }\\n}\\n\\n

output: - Tanaka-san receives an email from a client to reschedule a meeting with the boss.
- Boss is very important and we can\'t just simply change his time.
- Tanaka-san knows that the client is very important and we need to prioritize the client.
- Tanaka-san prioritizes the client. Per their request, Tanaka moved the meeting with them despite it clashes with another important meeting with the boss


input: # Action History\\n\\n## Action 1\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"Email client\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the email list.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client. The inbox is empty and a message shows \\\\\"What a productive day! You\\\\\\\\\\\\\\\'ve accomplished a lot.\\\\\"\"\\n    }\\n  }\\n}\\n\\n## Action 2\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"message\\\\\": \\\\\"What a productive day! You\\\\\\\\\\\\\\\'ve accomplished a lot\\\\\", \\\\\"status\\\\\": \\\\\"Syncing...\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"Email client showing a \\\\\"What a productive day!\\\\\" message.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor hovers over the empty right panel of the email client.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client. It displays a congratulatory message about a productive day with a hot air balloon illustration. The right panel appears empty.\"\\n    }\\n  }\\n}\\n\\n## Action 3\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"No emails are displayed.\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"The email client is open and displaying a message indicating there are no new emails.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse is not doing anything.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client. It looks like there are no unread emails. The email client says \\\\\"What a productive day! You\\\\\\\\\\\\\\\'ve accomplished a lot.\\\\\"\"\\n    }\\n  }\\n}\\n\\n## Action 4\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"Email client\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse is not doing anything.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client with no emails in the inbox. It says \\\\\"What a productive day! You\\\\\\\\\\\\\\\'ve accomplished a lot\\\\\".\"\\n    }\\n  }\\n}\\n\\n## Action 5\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"subject\\\\\": \\\\\"Rescheduling Meeting\\\\\", \\\\\"sender\\\\\": \\\\\"Fujitsu\\\\\", \\\\\"content\\\\\": \\\\\"Hi, Can we reschedule our meeting one hour later, from 2024-11-17 1...\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"Email client displaying a new email.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the email body.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client with a new email from Fujitsu about rescheduling a meeting. The user is reading the email.\"\\n    }\\n  }\\n}\\n\\n## Action 6\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"sender\\\\\": \\\\\"Fujitsu\\\\\", \\\\\"subject\\\\\": \\\\\"Rescheduling Meeting\\\\\", \\\\\"content\\\\\": \\\\\"Hi, Can we reschedule our meeting one hour later, from 2024-11-17 1...\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"Email client displaying a new email.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the email message.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client with a new email from Fujitsu regarding rescheduling a meeting. The email suggests moving the meeting one hour later from the original time of 2024-11-17. The email client is currently syncing.\"\\n    }\\n  }\\n}\\n\\n## Action 7\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"subject\\\\\": \\\\\"Rescheduling Meeting\\\\\", \\\\\"sender\\\\\": \\\\\"Fujitsu <fujitsu.fujitsuxxx@gmail.com>\\\\\", \\\\\"recipient\\\\\": \\\\\"kta.yasuda@gmail.com\\\\\", \\\\\"content\\\\\": \\\\\"Hi,\\\\\\\\n\\\\\\\\nCan we reschedule our meeting one hour later,\\\\\\\\nfrom 2024-11-17 16:00~17:00 to\\\\\\\\n17:00~18:00?\\\\\\\\n\\\\\\\\nThank you,\\\\\\\\nFujitsu\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email client open to an email about rescheduling a meeting.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse is not interacting with the email client.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client with an open email. The email is about rescheduling a meeting. \"\\n    }\\n  }\\n}\\n\\n## Action 8\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"subject\\\\\": \\\\\"Rescheduling Meeting\\\\\", \\\\\"sender\\\\\": \\\\\"Fujitsu <fujitsu.fujitsuxxx@gmail.com>\\\\\", \\\\\"recipient\\\\\": \\\\\"kta.yasuda@gmail.com\\\\\", \\\\\"content\\\\\": \\\\\"Hi,\\\\\\\\\\\\\\\\n\\\\\\\\\\\\\\\\nCan we reschedule our meeting one hour later,\\\\\\\\\\\\\\\\nfrom 2024-11-17 16:00~17:00 to\\\\\\\\\\\\\\\\n17:00~18:00?\\\\\\\\\\\\\\\\n\\\\\\\\\\\\\\\\nThank you,\\\\\\\\\\\\\\\\nFujitsu\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email client open to a message about rescheduling a meeting.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the \\\\\"+\\\\\" button, likely to create a new email or perform another action.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client with an open email. The email is about rescheduling a meeting.\"\\n    }\\n  }\\n}\\n\\n## Action 9\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"date\\\\\": \\\\\"Sunday, November 17, 2024\\\\\", \\\\\"time\\\\\": \\\\\"16:00 to 17:00\\\\\", \\\\\"event_title\\\\\": \\\\\"Important Meeting with Boss\\\\\", \\\\\"highlighted\\\\\": true, \\\\\"conflict\\\\\": true}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"A calendar application displaying the week\\\\\\\\\\\\\\\'s schedule.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the \\\\\"Important Meeting with Boss\\\\\" event on Sunday, November 17th. It appears the user is about to click or just clicked on either \\\\\"Edit Event\\\\\" or \\\\\"Delete Event\\\\\".\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows a calendar application open on a desktop. The calendar is displaying the week of November 11th to 17th, 2024. The current date is Saturday, November 16th.  An event titled \\\\\"Important Meeting with Boss\\\\\" is scheduled for Sunday, November 17th, from 4 PM to 5 PM. Another event, \\\\\"Client Meeting: Fujitsu,\\\\\" is also scheduled for Sunday, seemingly overlapping with the \\\\\"Important Meeting with Boss\\\\\". A pop-up window displays details of the \\\\\"Important Meeting with Boss\\\\\" event, with options to \\\\\"Edit Event\\\\\" or \\\\\"Delete Event\\\\\".\"\\n    }\\n  }\\n}\\n\\n## Action 10\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"date\\\\\": \\\\\"November 17, 2024\\\\\", \\\\\"time\\\\\": \\\\\"16:00 to 17:00\\\\\", \\\\\"event_title\\\\\": \\\\\"Important Meeting with Boss\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"A calendar application displaying the week\\\\\\\\\\\\\\\'s schedule.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the \\\\\"Important Meeting with Boss\\\\\" event, specifically the instance labeled \\\\\"Client Meeting: Fujitsu.\\\\\" This suggests the user is viewing event details or potentially about to interact with it, perhaps to edit, move, or delete it.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows a calendar application open on a computer screen. The calendar is displaying the week of November 11th to 17th, 2024. The current date is Saturday, November 16th.  An event titled \\\\\"Important Meeting with Boss\\\\\" is scheduled for Sunday, November 17th, from 4 PM to 5 PM. Another event, \\\\\"Client Meeting: Fujitsu,\\\\\" also labeled \\\\\"Important Meeting with Boss,\\\\\" is displayed, potentially overlapping or adjacent to the first event. The user interface includes options for creating a new event, filtering, and changing the week view. The location is set to New York, NY, with the current weather displayed as 10°C. The current time is likely around 4 PM, although not explicitly shown on the calendar interface itself.\"\\n    }\\n  }\\n}\\n\\n## Action 11\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"date\\\\\": \\\\\"November 17, 2024\\\\\", \\\\\"time\\\\\": \\\\\"15:00-16:00\\\\\", \\\\\"title\\\\\": \\\\\"Client Meeting: Fujitsu\\\\\", \\\\\"conflicts\\\\\": [\\\\\"Important Meeting with Boss\\\\\"]}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"A calendar application displaying the week\\\\\\\\\\\\\\\'s schedule.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Mail\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"unknown\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: false\\n              }\\n            }\\n          }\\n        }\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"chat\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Chat\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"unknown\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: false\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the \\\\\"Important Meeting with Boss\\\\\" event, indicating the user is viewing its details.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows a calendar application open on a computer. The calendar is displaying the week of November 11th to 17th, 2024. The current date is Saturday, November 16th.  A pop-up window displays details of an event scheduled for Sunday, November 17th, titled \\\\\"Client Meeting: Fujitsu\\\\\" from 3 PM to 4 PM. Another event, \\\\\"Important Meeting with Boss,\\\\\" is scheduled for the same time. The location is set to New York, NY, and the weather is 10°C.\"\\n    }\\n  }\\n}\\n\\n## Action 12\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"date\\\\\": \\\\\"November 17, 2024\\\\\", \\\\\"time\\\\\": \\\\\"15:00-16:00\\\\\", \\\\\"title\\\\\": \\\\\"Client Meeting: Fujitsu\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"A calendar application displaying the week of November 11-17, 2024, with events scheduled.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over Monday, November 11th.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows a calendar application open on a computer. The calendar is displaying the week of November 11th to 17th, 2024. The current date is Saturday, November 16th.  A pop-up window displays details of an event scheduled for Sunday, November 17th, from 3 PM to 4 PM, titled \\\\\"Client Meeting: Fujitsu\\\\\". Another event is partially visible, labeled \\\\\"Important Meeting with Boss\\\\\".\"\\n    }\\n  }\\n}\\n\\n## Action 13\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"date\\\\\": \\\\\"November 17, 2024\\\\\", \\\\\"time\\\\\": \\\\\"15:00 to 16:00\\\\\", \\\\\"title\\\\\": \\\\\"Client Meeting: Fujitsu\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"A calendar application displaying the week\\\\\\\\\\\\\\\'s schedule.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The user is hovering over the \\\\\"Client Meeting: Fujitsu\\\\\" event details.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows a calendar application open on a desktop or laptop. The calendar displays the week of November 11th to 17th, 2024, with Saturday, November 16th highlighted.  A pop-up window displays details of an event scheduled for Sunday, November 17th: \\\\\"Client Meeting: Fujitsu\\\\\" from 15:00 to 16:00.  Additional events are visible, including \\\\\"Important Meeting with Boss.\\\\\" The current time is indicated by a red line at approximately 13:00 on Saturday.\"\\n    }\\n  }\\n}\\n\\n## Action 14\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"subject\\\\\": \\\\\"Rescheduling Meeting\\\\\", \\\\\"sender\\\\\": \\\\\"Fujitsu <fujitsu.fujitsuxxx@gmail.com>\\\\\", \\\\\"recipient\\\\\": \\\\\"kta.yasuda@gmail.com\\\\\", \\\\\"body\\\\\": \\\\\"Hi,\\\\\\\\n\\\\\\\\nCan we reschedule our meeting one hour later,\\\\\\\\\\\\\\\\nfrom 2024-11-17 16:00~17:00 to 17:00~18:00?\\\\\\\\\\\\\\\\n\\\\\\\\\\\\\\\\nThank you,\\\\\\\\\\\\\\\\nFujitsu\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email client open to an email requesting a meeting be rescheduled.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the \\\\\"New Mail\\\\\" button.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email open in an email client. The user is requesting to reschedule a meeting.\"\\n    }\\n  }\\n}\\n\\n## Action 15\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"sender\\\\\": \\\\\"Kamil Ya (kta.yasuda@gmail.com)\\\\\", \\\\\"recipient\\\\\": \\\\\"isseki\\\\\", \\\\\"subject\\\\\": \\\\\"What should I do??\\\\\", \\\\\"body\\\\\": \\\\\"Hi Boss,\\\\\\\\\\\\\\\\n\\\\\\\\\\\\\\\\nFujitsu client wants to move the meeting to 5pm, but we have very Important Meeting.\\\\\\\\\\\\\\\\n\\\\\\\\\\\\\\\\nWhat should I do?\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"The user is using an email client to write an email.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is not visible in the screenshot, so we don\\\\\\\\\\\\\\\'t know what action the user is doing with it.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The user is writing an email to their boss about rescheduling a meeting with a Fujitsu client. The client wants to move the meeting to 5pm, but there is already another important meeting scheduled at that time. The user is asking their boss for advice on what to do.\"\\n    }\\n  }\\n}\\n\\n## Action 16\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"sender\\\\\": \\\\\"Kamil Ya (kta.yasuda@gmail.com)\\\\\", \\\\\"recipient\\\\\": \\\\\"isseki\\\\\", \\\\\"subject\\\\\": \\\\\"What should I do??\\\\\", \\\\\"body\\\\\": \\\\\"Hi Boss,\\\\\\\\\\\\\\\\n\\\\\\\\\\\\\\\\nFujitsu client wants to move\\\\\\\\\\\\\\\\nthe meeting to 5pm, but we\\\\\\\\\\\\\\\\nhave very Important Meeting.\\\\\\\\\\\\\\\\n\\\\\\\\\\\\\\\\nWhat should I do?\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"Email client open with new email being composed\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the \\\\\"To\\\\\" field, likely after typing in the recipient\\\\\\\\\\\\\\\'s name.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client open with a new email being composed. The email is regarding rescheduling a meeting with a Fujitsu client. The user is seeking advice from their boss on how to proceed because they already have a very important meeting scheduled at the proposed new time.\"\\n    }\\n  }\\n}\\n\\n## Action 17\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"sender\\\\\": \\\\\"Kamil Ya (kta.yasuda@gmail.com)\\\\\", \\\\\"recipient\\\\\": \\\\\"isseki\\\\\", \\\\\"subject\\\\\": \\\\\"What should I do??\\\\\", \\\\\"body\\\\\": \\\\\"Hi Boss,\\\\\\\\\\\\\\\\n\\\\\\\\\\\\\\\\nFujitsu client wants to move\\\\\\\\\\\\\\\\nthe meeting to 5pm, but we\\\\\\\\\\\\\\\\nhave very Important Meeting.\\\\\\\\\\\\\\\\n\\\\\\\\\\\\\\\\nWhat should I do?\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"Email client with a new email being composed\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the \\\\\"Send\\\\\" button.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client open with a new email being composed. The user is asking their boss what they should do because the Fujitsu client wants to move the meeting to 5pm, but there is already a very important meeting at that time.\"\\n    }\\n  }\\n}\\n\\n## Action 18\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"subject\\\\\": \\\\\"Rescheduling Meeting\\\\\", \\\\\"sender\\\\\": \\\\\"Fujitsu <fujitsu.fujitsuxxx@gmail.com>\\\\\", \\\\\"recipient\\\\\": \\\\\"kta.yasuda@gmail.com\\\\\", \\\\\"content\\\\\": \\\\\"Hi,\\\\\\\\n\\\\\\\\nCan we reschedule our meeting one hour later, from 2024-11-17 16:00~17:00 to 17:00~18:00?\\\\\\\\n\\\\\\\\nThank you,\\\\\\\\nFujitsu\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email client.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The user is sending an email.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client with an email open. The email is about rescheduling a meeting.\"\\n    }\\n  }\\n}\\n\\n## Action 19\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"subject\\\\\": \\\\\"Rescheduling Meeting\\\\\", \\\\\"sender\\\\\": \\\\\"Fujitsu <fujitsu.fujitsuxxx@gmail.com>\\\\\", \\\\\"recipient\\\\\": \\\\\"kta.yasuda@gmail.com\\\\\", \\\\\"body\\\\\": \\\\\"Hi,\\\\\\\\\\\\\\\\nCan we reschedule our meeting one hour later,\\\\\\\\\\\\\\\\nfrom 2024-11-17 16:00~17:00 to 17:00~18:00?\\\\\\\\\\\\\\\\nThank you,\\\\\\\\\\\\\\\\nFujitsu\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"Email client displaying an email about rescheduling a meeting.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the \\\\\"Sync\\\\\" button, likely about to synchronize the email account.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot displays an email client, likely on a desktop or laptop computer. The email client shows an email from \\\\\"Fujitsu\\\\\" with the subject \\\\\"Rescheduling Meeting\\\\\". The email body requests to reschedule a meeting from 4-5 PM to 5-6 PM on November 17, 2024.\"\\n    }\\n  }\\n}\\n\\n## Action 20\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"email_threads\\\\\": [{\\\\\"sender\\\\\": \\\\\"isseki\\\\\", \\\\\"subject\\\\\": \\\\\"What should I do?\\\\\", \\\\\"content\\\\\": \\\\\"Sure, Fujitsu is a very important client, we should always put Fujitsu fi...\\\\\"}, {\\\\\"sender\\\\\": \\\\\"Fujitsu\\\\\", \\\\\"subject\\\\\": \\\\\"Rescheduling Meeting\\\\\", \\\\\"content\\\\\": \\\\\"Hi, Can we reschedule our meeting one hour later, from 2024-11-17 16:00~17:00 to 17:00~18:00?\\\\\"}]}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email client open to two email threads.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the first email thread.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client with two email threads. The first email thread is from \\\\\"isseki\\\\\" and asks \\\\\"What should I do?\\\\\". The second email thread is from \\\\\"Fujitsu\\\\\" and requests to reschedule a meeting. \"\\n    }\\n  }\\n}\\n\\n## Action 21\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"subject\\\\\": \\\\\"Re: What should I do??\\\\\", \\\\\"sender\\\\\": \\\\\"isseki <isseki.ni.to@gmail.com>\\\\\", \\\\\"recipient\\\\\": \\\\\"Kamil Ya\\\\\", \\\\\"content\\\\\": \\\\\"Sure, Fujitsu is a very important client, we should always put Fujitsu first. Feel free to reschedule our Very Important Meeting\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email client open to a conversation.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the email address kta.yasuda@gmail.com.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email conversation about rescheduling a meeting with an important client, Fujitsu. \"\\n    }\\n  }\\n}\\n\\n## Action 22\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"subject\\\\\": \\\\\"Re: What should I do??\\\\\", \\\\\"sender\\\\\": \\\\\"isseki <isseki.ni.to@gmail.com>\\\\\", \\\\\"recipient\\\\\": \\\\\"Kamil Ya\\\\\", \\\\\"content\\\\\": \\\\\"Sure, Fujitsu is a very important client, we should always put Fujitsu first. Feel free to reschedule our Very Important Meeting\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email client open to an inbox and displaying an email thread.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the list of emails in the inbox.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email thread in an email client. The user is viewing an email chain about rescheduling a meeting with a client, Fujitsu. \"\\n    }\\n  }\\n}\\n\\n## Action 23\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"subject\\\\\": \\\\\"Re: What should I do??\\\\\", \\\\\"sender\\\\\": \\\\\"isseki <isseki.ni.to@gmail.com>\\\\\", \\\\\"recipient\\\\\": \\\\\"Kamil Ya\\\\\", \\\\\"content\\\\\": \\\\\"Sure, Fujitsu is a very important client, we should always put Fujitsu first. Feel free to reschedule our Very Important Meeting\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email client displaying an email conversation.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the email from \\\\\"isseki\\\\\" with the subject \\\\\"What should I do?\\\\\".\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email conversation between isseki and Kamil Ya about rescheduling a meeting with a client, Fujitsu. \"\\n    }\\n  }\\n}\\n\\n## Action 24\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"date\\\\\": \\\\\"November 16, 2024\\\\\", \\\\\"events\\\\\": [{\\\\\"time\\\\\": \\\\\"13:01\\\\\", \\\\\"title\\\\\": \\\\\"Client Meeting: Fujitsu\\\\\"}, {\\\\\"time\\\\\": \\\\\"Not specified\\\\\", \\\\\"title\\\\\": \\\\\"Important Meeting with Boss\\\\\"}]}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"A calendar application displaying the week\\\\\\\\\\\\\\\'s schedule.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the \\\\\"Client Meeting: Fujitsu\\\\\" entry.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows a calendar application open on a computer screen. The current week is displayed, showing the days from Monday, November 11th to Sunday, November 17th. The current date is Saturday, November 16th, and the time is 13:01.  Two meetings are scheduled for Saturday: \\\\\"Client Meeting: Fujitsu\\\\\" and \\\\\"Important Meeting with Boss\\\\\".\"\\n    }\\n  }\\n}\\n\\n## Action 25\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"date\\\\\": \\\\\"November 16, 2024\\\\\", \\\\\"events\\\\\": [{\\\\\"title\\\\\": \\\\\"Client Meeting: Fujitsu\\\\\"}, {\\\\\"title\\\\\": \\\\\"Important Meeting with Boss\\\\\"}]}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"A calendar application displaying the week\\\\\\\\\\\\\\\'s schedule.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the \\\\\"Client Meeting: Fujitsu\\\\\" entry.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows a calendar application open on a computer. The calendar is displaying the week of November 11th to 17th, 2024.  The current date is Saturday, November 16th. The location is set to New York, NY, and the weather is 10°C. Two meetings are scheduled for Saturday: one labeled \\\\\"Client Meeting: Fujitsu\\\\\" and another labeled \\\\\"Important Meeting with Boss\\\\\".\"\\n    }\\n  }\\n}\\n\\n## Action 26\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"date\\\\\": \\\\\"November 16, 2024\\\\\", \\\\\"entries\\\\\": [\\\\\"Client Meeting: Fujitsu\\\\\", \\\\\"Client Meeting: Fujitsu\\\\\", \\\\\"Important Meeting with Boss\\\\\"]}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"A calendar application displaying the week view.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"selecting an item\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows a calendar app open on a computer screen. The calendar is set to week view, displaying the week of November 11th to 17th. The current date is Saturday, November 16th.  The user has multiple entries for Saturday, and appears to be selecting one of them, an \\\\\"Important meeting with Boss\\\\\"\"\\n    }\\n  }\\n}\\n\\n## Action 27\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"date\\\\\": \\\\\"Saturday, November 16th\\\\\", \\\\\"time\\\\\": \\\\\"16:00\\\\\", \\\\\"title\\\\\": \\\\\"Client Meeting: Fujitsu\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"A calendar application displaying the week\\\\\\\\\\\\\\\'s schedule.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"Hovering over an event\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows a calendar application open on a computer. The current week being displayed is November 11th to 17th, with today, Saturday the 16th highlighted.  An event is scheduled for today, starting at 16:00, titled \\\\\"Client Meeting: Fujitsu\\\\\". The mouse cursor is hovering over this event, likely about to click on it or interact with it in some way.\"\\n    }\\n  }\\n}\\n\\n## Action 28\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"date\\\\\": \\\\\"November 16th\\\\\", \\\\\"events\\\\\": [{\\\\\"name\\\\\": \\\\\"Client Meeting\\\\\", \\\\\"description\\\\\": \\\\\"Fujitsu\\\\\"}, {\\\\\"name\\\\\": \\\\\"Important Meeting\\\\\", \\\\\"description\\\\\": \\\\\"with\\\\\"}]}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"A calendar application displaying the week view.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the \\\\\"Important Meeting with\\\\\" entry on Saturday.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows a calendar application open on a computer. The calendar is set to week view, displaying the dates from November 11th to 17th. The current date, Saturday, November 16th, is highlighted.  There are two entries scheduled for Saturday: \\\\\"Client Meeting: Fujitsu\\\\\" and \\\\\"Important Meeting with\\\\\". The current time is 13:01.\"\\n    }\\n  }\\n}\\n\\n## Action 29\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"date\\\\\": \\\\\"November 16, 2024\\\\\", \\\\\"events\\\\\": [{\\\\\"time\\\\\": \\\\\"unspecified\\\\\", \\\\\"title\\\\\": \\\\\"Client Meeting: Fujitsu\\\\\"}, {\\\\\"time\\\\\": \\\\\"unspecified\\\\\", \\\\\"title\\\\\": \\\\\"Important Meeting with\\\\\"}, {\\\\\"time\\\\\": \\\\\"unspecified\\\\\", \\\\\"title\\\\\": \\\\\"Meeting with\\\\\"}]}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"A calendar application.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The user is clicking and dragging to resize one of the meetings scheduled for Saturday.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows a calendar application open to the week of November 11-17. The current date is Saturday, November 16th.  The user has several meetings scheduled, including two overlapping meetings both labeled \\\\\"Meeting with\\\\\" but with the titles cut off.\"\\n    }\\n  }\\n}\\n\\n## Action 30\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"date\\\\\": \\\\\"November 16th\\\\\", \\\\\"time\\\\\": \\\\\"Evening\\\\\", \\\\\"events\\\\\": [{\\\\\"title\\\\\": \\\\\"Meeting with Client: Fujitsu\\\\\", \\\\\"highlighted\\\\\": true, \\\\\"conflict\\\\\": false}]}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"A calendar application displaying the week view.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The user is clicking and dragging the mouse to create a new event on the calendar.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows a calendar application open on a computer. The calendar is set to week view, displaying the week of November 11th to 17th. The current date is Saturday, November 16th.  The location is set to New York, NY, and the weather is 10°C. There are several scheduled events, one of which is being created or edited. The user is creating a new event by clicking and dragging on the calendar grid. The new event is scheduled for Saturday the 16th, starting sometime after 6 PM. The title of the event is not yet fully visible, but it includes \\\\\"Meeting\\\\\", \\\\\"Client\\\\\", and possibl
output:"""

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

generate()

# Thinking
* **Actions 1-4:** The employee opens their email client. The inbox is empty, and the client is syncing.  The employee likely checks their email regularly to stay on top of communications.
* **Action 5-8:** An email arrives from Fujitsu requesting to reschedule a meeting. The employee opens and reads the email carefully, noting the proposed new time.
* **Action 9-13:** The employee opens their calendar application to check the existing schedule for the day the meeting is supposed to take place. They identify a scheduling conflict with another important meeting, "Important Meeting with Boss."  They examine the details of both meetings.
* **Action 14-17:** Recognizing the conflict and needing guidance, the employee composes an email to their boss ("isseki") to explain the situation and ask for instructions on how to proceed. The email subject is "What should I do??" They hover over the "send" button, likely feeling some stress about the situation.
* **Action 18-19:** The emplo

In [87]:
action_history_example = """# Action History\\n\\n## Action 1\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"status\\\\\": \\\\\"Syncing\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email client application.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the right panel.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client application. The inbox is empty, and a message displays \\\\\"What a productive day! You\\\\\\\\\\\\\\\'ve accomplished a lot\\\\\". The client is currently syncing.\"\\n    }\\n  }\\n}\\n\\n## Action 2\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Mail\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"Email client application\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse is idle.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client application. The inbox is empty and displays a congratulatory message: \\\\\"What a productive day! You\\\\\\\\\\\\\\\'ve accomplished a lot\\\\\". The application is currently syncing.\"\\n    }\\n  }\\n}\\n\\n## Action 3\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Mail\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"Email client open with no emails present, showing a congratulatory message about having a productive day.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the \\\\\"Sync\\\\\" button.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client with no emails in the inbox. The email client displays a hot air balloon image and the text \\\\\"What a productive day! You\\\\\\\\\\\\\\\'ve accomplished a lot\\\\\".\"\\n    }\\n  }\\n}\\n\\n## Action 4\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"sender\\\\\": \\\\\"Fujitsu\\\\\", \\\\\"subject\\\\\": \\\\\"Rescheduling Meeting\\\\\", \\\\\"content\\\\\": \\\\\"Hi, Can we reschedule our meeting one hour later, from 2024-11-17 1...\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email from Fujitsu regarding rescheduling a meeting.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"Hovering over the Sync button\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client with a request to reschedule a meeting. The user is about to click the Sync button.\"\\n    }\\n  }\\n}\\n\\n## Action 5\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"sender\\\\\": \\\\\"Fujitsu\\\\\", \\\\\"subject\\\\\": \\\\\"Rescheduling Meeting\\\\\", \\\\\"content\\\\\": \\\\\"Hi, Can we reschedule our meeting one hour later, from 2024-11-17 1...\\\\\", \\\\\"time\\\\\": \\\\\"12:59\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email client displaying a message about rescheduling a meeting.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"Hovering over the email content.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client. The user is reading an email about rescheduling a meeting.\"\\n    }\\n  }\\n}\\n\\n## Action 6\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"email\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Email Client\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"subject\\\\\": \\\\\"Rescheduling Meeting\\\\\", \\\\\"sender\\\\\": \\\\\"Fujitsu <fujitsu.fujitsuxxx@gmail.com>\\\\\", \\\\\"recipient\\\\\": \\\\\"kta.yasuda@gmail.com\\\\\", \\\\\"body\\\\\": \\\\\"Hi,\\\\\\\\\\\\\\\\n\\\\\\\\\\\\\\\\nCan we reschedule our meeting one hour later,\\\\\\\\\\\\\\\\nfrom 2024-11-17 16:00~17:00 to\\\\\\\\\\\\\\\\n17:00~18:00?\\\\\\\\\\\\\\\\n\\\\\\\\\\\\\\\\nThank you,\\\\\\\\\\\\\\\\nFujitsu\\\\\"}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"An email client open to a message thread.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the email body text.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows an email client with an email thread open. The user is reading an email about rescheduling a meeting.\"\\n    }\\n  }\\n}\\n\\n## Action 7\\nname: \"screenshot_data\"\\nargs {\\n  fields {\\n    key: \"programs_in_scene\"\\n    value {\\n      list_value {\\n        values {\\n          struct_value {\\n            fields {\\n              key: \"program_type\"\\n              value {\\n                string_value: \"calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_name\"\\n              value {\\n                string_value: \"Calendar\"\\n              }\\n            }\\n            fields {\\n              key: \"program_info\"\\n              value {\\n                string_value: \"{\\\\\"date\\\\\": \\\\\"November 16, 2024\\\\\", \\\\\"events\\\\\": [{\\\\\"time\\\\\": \\\\\"13:01\\\\\", \\\\\"title\\\\\": \\\\\"Client Meeting: Fujitsu\\\\\"}, {\\\\\"time\\\\\": \\\\\"13:01\\\\\", \\\\\"title\\\\\": \\\\\"Important Meeting with Boss\\\\\"}]}\"\\n              }\\n            }\\n            fields {\\n              key: \"program_description\"\\n              value {\\n                string_value: \"A calendar application.\"\\n              }\\n            }\\n            fields {\\n              key: \"is_active\"\\n              value {\\n                bool_value: true\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n  fields {\\n    key: \"mouse_action\"\\n    value {\\n      string_value: \"The mouse cursor is hovering over the \\\\\"Client Meeting: Fujitsu\\\\\" entry.\"\\n    }\\n  }\\n  fields {\\n    key: \"general_scene_description\"\\n    value {\\n      string_value: \"The screenshot shows a calendar application open on a computer. The calendar is displaying the week of November 11th to 17th, 2024.  The current date is Saturday, November 16th. Two meetings are scheduled for Saturday: one with Fujitsu and another, marked as important, with the user\\\\\\\\\\\\\\\'s boss. The current time is 1:01 PM. The location is set to New York, NY, and the weather is 10°C (feels like 16°C) with a low of 7°C.\"\\n    }\\n  }\\n}\\n\\n"""

In [88]:
def get_prompt_for_summarizer(action_history):
    text1 = f"""An employee receives an email from a client requesting to change the schedule. The employee goes through a series of actions to complete that task. `# Action History` holds the list of activities for executing this task. Some information in the activity list is not obvious to the newcomers. 

    Your task is to comprehend each activity and infer the abstract overview concept, capturing what each task is about, and why each task is performed. Find a series of actions of what has been done. Then, infer the non-obvious in-depth as to WHY each action is taken.  

    Extract the reasoning behind each action and decision in bullet points. Think step by step and write it in the #Thinking section. Finally, comprehend key important lessons from this interaction in the #Lesson section.

    input: {action_history_example}

    output: - Tanaka-san receives an email from a client to reschedule a meeting with the boss.
    - Boss is very important and we can\'t just simply change his time.
    - Tanaka-san knows that the client is very important and we need to prioritize the client.
    - Tanaka-san prioritizes the client. Per their request, Tanaka moved the meeting with them despite it clashes with another important meeting with the boss


    input: {action_history}
    output:"""

    return text1

In [90]:
action_history = actions

In [95]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting

def generate(prompt):
    model = genai.GenerativeModel(
        "gemini-1.5-flash-002",
        system_instruction=["""Your task is to understand how an employee executes a task."""]
    )

    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }
        
    responses = model.generate_content(
        [prompt],
        generation_config=generation_config,
        stream=False,
    )

    output = ""
    for response in responses:
        # print(len(responses))
        # print(response.text, end="")
        output += response.text

    return output


generate(get_prompt_for_summarizer(action_history))

'# Thinking\n\n* **Action 1:** The employee opens their email client. The inbox is empty, indicating no new emails requiring immediate attention.  The "What a productive day!" message suggests the employee has completed tasks. The mouse hovering over the email list implies they are checking for any new messages.\n\n* **Action 2:**  The employee is still in the email client, and the mouse is idle. The syncing message suggests the email client is updating, likely in anticipation of new messages.\n\n* **Action 3:** The mouse cursor hovers over the "Sync" button. This action shows the employee is actively waiting for their inbox to update.  They are likely expecting an email or have a reason to believe one is imminent.\n\n* **Action 4:** A new email appears in the inbox from Fujitsu regarding rescheduling a meeting. The employee\'s mouse is still over the Sync button, implying they saw this email appear after syncing.\n\n* **Action 5:** The employee hovers over the email content. This show

In [93]:
responses

NameError: name 'responses' is not defined